### 수행목표
- 랜덤 추천의 결과를 통해 회귀 평가를 계산하는 기능을 개발한다.
### 수행단계
- RandomRecommender.run()에서 return한 결과를 통해 random 평점이 결과(rate)와 얼마나 비슷한지 확인하는 코드를 만든다.
- 결과를 분석하는 class 명을 Analyzer로 하고 결과 데이터를 입력하여 분석 결과를 return 한다.
- 평균 절대 오차 (Mean Absolute Error - MAE)를 계산한다.
- 평균 제곱 오차 (Mean Squared Error - MSE)를 계산한다.
- 평균 제곱근 오차 (Root Mean Squared Error - RMSE)를 계산한다.
- 평균 절대 비율 오차 (Mean Absolute Percentage Error - MAPE)를 계산한다.
결과 데이터 평점 높은 순 n개를 출력하고 그 아래에 분석 결과를 출력한다.


In [13]:
# MAE (Mean Absolute Error, 평균 절대 오차)  
# 예측값과 실제값 간의 차이를 절대값으로 계산한 뒤 평균을 구한 값. 오차의 크기를 직관적으로 보여줌. 이상치(outlier)에 민감하지 않으며, 값이 낮을수록 모델의 전반적인 예측 정확도가 높음을 의미.

# MSE (Mean Squared Error, 평균 제곱 오차) 
# 예측값과 실제값 간의 차이를 제곱한 뒤 평균을 구한 값. 큰 오차에 더 큰 영향을 부여함. 이상치에 민감하며, 값이 낮을수록 모델이 큰 오차를 잘 줄였음을 나타냄.

# RMSE (Root Mean Squared Error, 평균 제곱근 오차)  
# MSE의 제곱근을 취한 값. MSE와 동일하게 큰 오차에 민감하지만, 데이터와 동일한 단위를 가지므로 해석이 더 직관적임. 값이 낮을수록 모델의 성능이 우수함을 의미.

# MAPE (Mean Absolute Percentage Error, 평균 절대 비율 오차) 
# 예측값과 실제값 간의 차이를 실제값으로 나눠 비율로 계산한 뒤 평균을 구한 값. 오차를 상대적인 퍼센트로 표현하므로 해석이 쉬움. 데이터에 0이 포함되면 계산이 왜곡될 수 있음. 값이 낮을수록 예측이 실제값에 비례해 정확함을 나타냄.


### 결론
# - 랜덤 추천은 임의로 영화에 무작위 평점을 부여하고 이를 기준으로 상위 영화를 추천하는 방식으로 이루어짐.  
# - 4종 평가는 무작위로 부여된 평점(`rate_random`)과 실제 평점(`rate`) 간의 차이를 정량화하여 지표로 나타냄.  
# - 이 지표들은 랜덤 추천이 잘 이루어졌는지, 잘못되었는지를 판단하는 도구가 아님.  
# - 단순히 무작위 평점과 실제 평점 사이의 정량적 차이를 수치로 보여주는 역할을 함.  
# - 4종 평가는 랜덤 추천의 성능 평가보다는 다른 추천 알고리즘과 비교하거나 모델의 기준선을 제공하는 데 사용됨.  

In [14]:
import pandas as pd
import numpy as np

# 데이터를 load() 하고 전처리하는 class
class MovieDataLoader:
    def __init__(self, movies_file: str, rates_file: str):
        # 영화 데이터 파일 경로와 평점 데이터 파일 경로를 저장
        self.movies_file = movies_file
        self.rates_file = rates_file

    def load(self):
        # 영화 데이터(movies.txt) 파일을 로드
        # '\t'로 구분된 데이터를 Pandas DataFrame으로 읽어옴
        # 'movie' 컬럼을 인덱스로 설정
        movies = pd.read_csv(self.movies_file, sep="\t", engine="python", index_col="movie")
        
        # 평점 데이터(rates.csv) 파일을 로드
        rates = pd.read_csv(self.rates_file)
        
        # 로드된 데이터프레임 반환
        return movies, rates


# 추천 코드 class
class RandomRecommender:
    def __init__(self, movies, rates):
        # 생성자에서 영화 데이터와 평점 데이터를 받아서 저장
        self.movies = movies
        self.rates = rates

    def run(self, n=10):
        # Numpy를 활용해 랜덤 평점(rate_random) 생성
        # random.uniform은 부동소수점(0~10) 값을 반환하며 np.round로 소수점 2자리까지 반올림
        self.rates['rate_random'] = np.round(np.random.uniform(0, 10, size=len(self.rates)), 2)

        # 영화 데이터와 평점 데이터를 병합
        # 'movie' 컬럼을 기준으로 병합하여 각 평점에 영화 정보를 연결
        merged_data = pd.merge(self.rates, self.movies, left_on='movie', right_index=True)

        # 'rate_random' 컬럼을 기준으로 내림차순 정렬
        sorted_data = merged_data.sort_values(by="rate_random", ascending=False)

        # 상위 n개의 데이터 반환
        return sorted_data.head(n)

In [15]:
# 분석 class
class Analyzer:
    def __init__(self, results):
        # 분석할 결과 데이터를 저장
        self.results = results

    def calculate_metrics(self):
        # 실제 평점과 랜덤 평점을 분리
        actual = self.results['rate']  # 실제 평점
        predicted = self.results['rate_random']  # 랜덤으로 생성된 평점

        # MAE: Mean Absolute Error (평균 절대 오차)
        mae = np.mean(np.abs(actual - predicted))

        # MSE: Mean Squared Error (평균 제곱 오차)
        mse = np.mean((actual - predicted) ** 2)

        # RMSE: Root Mean Squared Error (평균 제곱근 오차)
        rmse = np.sqrt(mse)

        # MAPE: Mean Absolute Percentage Error (평균 절대 비율 오차)
        # 실제 값이 0인 경우 계산이 불가능하므로 이를 방지
        mape = np.mean(np.abs((actual - predicted) / actual.replace(0, np.nan))) * 100

        # 결과를 딕셔너리 형태로 반환하며, 각 메트릭의 의미를 명시적으로 포함
        return {
            "MAE(평균 절대 오차)": mae,
            "MSE(평균 제곱 오차)": mse,
            "RMSE(평균 제곱근 오차)": rmse,
            "MAPE(평균 절대 비율 오차)": mape,
        }

In [17]:
# 메인 실행부
if __name__ == "__main__":
    movies_file = "kmrd-small/movies.txt"  # 영화 데이터 파일 경로
    rates_file = "kmrd-small/rates.csv"  # 평점 데이터 파일 경로

    # 데이터 로드
    loader = MovieDataLoader(movies_file, rates_file)
    movies, rates = loader.load()

    # 랜덤 추천 실행
    recommender = RandomRecommender(movies, rates)
    top_recommendations = recommender.run(n=5)

    # 결과 데이터 출력
    print("-------------------------")
    print("-----결과 데이터 가공-----") 
    print("-------------------------")
    result = top_recommendations[['rate_random', 'movie', 'rate', 'title', 'year']]
    print(result)

    print()
    print()
    
    # 분석 수행
    analyzer = Analyzer(result)
    metrics = analyzer.calculate_metrics()

    # 분석 결과 출력
    print("-------------------------")
    print("--------분석 결과--------")
    print("-------------------------")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")


-------------------------
-----결과 데이터 가공-----
-------------------------
       rate_random  movie  rate                 title    year
74932         10.0  10936     2                   가위손  2014.0
58034         10.0  10006    10  스타워즈 에피소드 5 - 제국의 역습  1997.0
60047         10.0  10037     9                  에이리언  1987.0
94448         10.0  10048    10             죽은 시인의 사회  2016.0
20323         10.0  10047     8                    스팅  1978.0


-------------------------
--------분석 결과--------
-------------------------
MAE(평균 절대 오차): 2.20
MSE(평균 제곱 오차): 13.80
RMSE(평균 제곱근 오차): 3.71
MAPE(평균 절대 비율 오차): 87.22
